In [20]:
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
from geemap.conversion import *
import ee
import folium
from folium import plugins
import datetime as dt

In [2]:
ee.Authenticate()

Enter verification code: 4/1ARtbsJqb_p-6BH2XqTurLZgvT9QSZTiRWfAvoV1ZUsGhH3CMQARt_ikNhUs

Successfully saved authorization token.


In [3]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
cart_classifier = ee.FeatureCollection("projects/ee-gupei/assets/randomforest_classifier_3")


geometry = ee.FeatureCollection("projects/ee-gupei/assets/Final_data_9k")

In [5]:
geometry

In [6]:
cart_classifier.first()


In [7]:
Sentinel2A = ee.ImageCollection("COPERNICUS/S2_SR");


In [8]:
classifier_string = cart_classifier.first().get('classifier');


In [9]:
classifier = ee.Classifier.decisionTree(classifier_string);


In [10]:
BANDS = ['B2', 'B3', 'B4', 'B8'];


In [37]:
from dateutil.relativedelta import *

now = dt.datetime.now()
later = now +relativedelta(months=+1)


ic = Sentinel2A.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
ic = ic.select(BANDS).filterDate(now, later).median();


In [12]:
classified = ic.classify(classifier);


In [13]:
classified.clip(geometry)


In [14]:
palette = [
  '0000FF', #Water
  '008000', #Veg
  'A52A2A'  #Land
]

In [15]:
lat = -37.8
lon = 145.2
Map = folium.Map(location=[lat, lon], zoom_start=10)

def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
folium.Map.add_ee_layer = add_ee_layer

In [16]:
Map
